In [ ]:
!pip install -U transformers datasets peft accelerate sentencepiece


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 18.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
  Attempting uninstall: peft
    Found existing installation: peft 0.17.1
    Uninstalling peft-0.17.1:
      Successfully uninstalled peft-0.17.1


In [ ]:
from datasets import load_dataset

ds = load_dataset("Amod/mental_health_counseling_conversations")

ds = ds["train"].train_test_split(test_size=0.2, seed=42)
ds["eval"] = ds["test"]
del ds["test"]

print("Train samples:", len(ds["train"]))
print("Eval samples:", len(ds["eval"]))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

combined_dataset.json: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/3512 [00:00<?, ? examples/s]

Train samples: 2809
Eval samples: 703


In [ ]:
def format_chat(example):
    user = example["Context"].strip()
    assistant = example["Response"].strip()
    example["text"] = f"<|user|>\n{user}\n<|assistant|>\n{assistant}"
    return example

ds["train"] = ds["train"].map(format_chat)
ds["eval"]  = ds["eval"].map(format_chat)



Map:   0%|          | 0/2809 [00:00<?, ? examples/s]

Map:   0%|          | 0/703 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_fn(batch):
    tokens = tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized = ds.map(
    tokenize_fn,
    batched=True,
    remove_columns=["Context", "Response", "text"]
)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Map:   0%|          | 0/2809 [00:00<?, ? examples/s]

Map:   0%|          | 0/703 [00:00<?, ? examples/s]

In [ ]:
import torch
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16
).to("cuda")

print("Model loaded on one GPU ✔")


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Model loaded on one GPU ✔


In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 4,505,600 || all params: 1,104,553,984 || trainable%: 0.4079


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="tinyllama-lora",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-5,
    num_train_epochs=1,
    logging_steps=50,
    logging_strategy="steps",
    eval_strategy = "steps",
    eval_steps=50, 
    fp16=True,
    save_steps=200,
    report_to="none"
)


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["eval"]
)


In [ ]:
trainer.train()


Step,Training Loss,Validation Loss
50,1.346000,1.320737
100,1.323400,1.310929
150,1.346000,1.306556
200,1.239700,1.303433
250,1.291100,1.301032
300,1.276800,1.299356
350,1.241200,1.297829
400,1.361500,1.296644
450,1.277300,1.295690
500,1.248300,1.294712


TrainOutput(global_step=703, training_loss=1.2913115095786993, metrics={'train_runtime': 1384.7394, 'train_samples_per_second': 2.029, 'train_steps_per_second': 0.508, 'total_flos': 8965943860396032.0, 'train_loss': 1.2913115095786993, 'epoch': 1.0})

In [ ]:
model.save_pretrained("tinyllama-lora-final")
tokenizer.save_pretrained("tinyllama-lora-final")
print("Model + tokenizer saved ✔")


Model + tokenizer saved ✔
